# OGGM glacier climate simulator

App to initialize a simple glacier and look how it develops under changing climate settings.

### Import packages

### Panel for intializing a simple glacier

should contain geometry of the glacier bed, ...

### Panel for modifying mass balance profile

change ELA and gradient, as a climate parameter, later maybe change temperature and precipitation and convert this changes into a mass balance profile

### Create plot for mass balance profile (dynamic plot)

showing current mass balance profile

### Create plot for glacier height (dynamic plot)

showing surface glacier height, maybe also from the previous step,...

### Put Panels and plots together